In [2]:
from pyk4a import PyK4A
import cv2
import helpers
import numpy as np
import k4a
import pyk4a
import ctypes
import  open3d
import sys
import os

In [13]:
os.getcwd()
os.chdir("D:")
os.getcwd()
#os.mkdir("storage")
os.chdir("D:\storage")
os.getcwd()
os.listdir("D:\storage")

['output.mkv']

In [3]:
# https://github.com/ibaiGorordo/pyKinectAzure/blob/226ab1cfb1d96b78a8fa38b260467e1fd04df2b9/pykinect_azure/utils/plot3dUtils.py#L1
import numpy as np
import cv2
import open3d as o3d

class Open3dVisualizer():

	def __init__(self):

		self.point_cloud = o3d.geometry.PointCloud()
		self.o3d_started = False

		self.vis = o3d.visualization.Visualizer()
		self.vis.create_window()

	def __call__(self, points_3d, rgb_image=None):

		self.update(points_3d, rgb_image)

	def update(self, points_3d, rgb_image=None):

		# Add values to vectors
		self.point_cloud.points = o3d.utility.Vector3dVector(points_3d)
		if rgb_image is not None:
			colors = cv2.cvtColor(rgb_image, cv2.COLOR_BGRA2RGB).reshape(-1,3)/255
			self.point_cloud.colors = o3d.utility.Vector3dVector(colors)

		self.point_cloud.transform([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])

		# Add geometries if it is the first time
		if not self.o3d_started:
			self.vis.add_geometry(self.point_cloud)
			self.o3d_started = True

		else:
			self.vis.update_geometry(self.point_cloud)

		self.vis.poll_events()
		self.vis.update_renderer()

In [10]:
# Testing pyk4a store and open3d
open3dViz = Open3dVisualizer()
k4a = PyK4A()
k4a.start()
i = 0
while 1:
    capture = k4a.get_capture()
    if np.any(capture):
        pc = capture.depth_point_cloud
        #np.save("pc_"+str(i), pc)
        i = i +1

        #out.write(capture.color)
        cv2.imshow("k4a color", capture.color[:, :, :3])
        cv2.imshow("k4a depth", helpers.colorize(capture.depth, (None, 5000), cv2.COLORMAP_HSV))
        cv2.imshow("k4a ir", capture.ir)
        key = cv2.waitKey(10)
        if key != -1:
            cv2.destroyAllWindows()
            break
k4a.stop()
print(pc)
#open3dViz.vis(pc)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
# Testing k4a library
device = k4a.k4a_device_t()
k4a.k4a_device_open(k4a.K4A_DEVICE_DEFAULT, device)
config = k4a.k4a_device_configuration_t
config = k4a.K4A_DEVICE_CONFIG_INIT_DISABLE_ALL
config.camera_fps = k4a.K4A_FRAMES_PER_SECOND_15
config.color_format = k4a.K4A_IMAGE_FORMAT_COLOR_BGRA32
config.color_resolution = k4a.K4A_COLOR_RESOLUTION_3072P

#k4a.k4a_device_start_cameras(device, config)

if (k4a.K4A_RESULT_SUCCEEDED != k4a.k4a_device_start_cameras(device, config)):
    print("Failed to start Device")


k4a.k4a_device_close(device)
# frame_count = 0
# while frame_count < 100:
#     sensor_capture = k4a.k4a_capture_t()
#     get_capture_result = k4a.k4a_device_get_capture(device, ctypes.byref(sensor_capture), k4a.K4A_WAIT_INFINITE)
#
#     if get_capture_result == k4a.K4A_WAIT_RESULT_SUCCEEDED:
#         frame_count += 1
#         cv2.imshow("test", get_capture_result.imag)
#         key = cv2.waitKey(10)
#         if key != -1:
#             cv2.destroyWindows()
#             break


Failed to start Device


17

In [15]:
k4a = PyK4A()
k4a.start()
#out = cv2.VideoWriter("text.avi", cv2.VideoWriter_fourcc('M','J','P','G'), 10, (1920,1080))
while 1:
    capture = k4a.get_capture()
    if np.any(capture):
        #out.write(capture.color)
        depth = capture.depth_point_cloud
        cv2.imshow("k4a color", capture.color[:, :, :3])
        cv2.imshow("k4a depth", helpers.colorize(capture.depth, (None, 5000), cv2.COLORMAP_HSV))
        cv2.imshow("k4a ir", capture.ir)
        key = cv2.waitKey(10)
        if key != -1:
            cv2.destroyAllWindows()
            break
k4a.stop()
#open3dViz(depth)
print(depth.ndim)


3


In [18]:
import os
import subprocess

if not os.path.exists("assets"):
    raise Exception("Please create and put all your vidoes in assets folder!")

mkv_list = os.listdir("assets")

if not os.path.exists("result"):
    os.mkdir("result")

for mkv in mkv_list:
    name, ext = os.path.splitext(mkv)
    if ext != ".mkv":
        raise Exception("Please add MKV files only!")

    output_name = name + ".mp4"
    try:
        subprocess.run(
            ["ffmpeg", "-i", f"assets/{mkv}", "-codec", "copy", f"result/{output_name}"], check=True
        )
    except:
        raise Exception(
            "Please DOWNLOAD, INSTALL & ADD the path of FFMPEG to Environment Variables!"
        )


print(f"{len(mkv_list)} video(s) converted to MP4!")
os.startfile("result")

1 video(s) converted to MP4!


In [2]:
import ffmpeg

In [9]:
input = ffmpeg.input("assets/output.mkv")

In [12]:
output = ffmpeg.output(input["2"], "testout.mkv")

In [1]:
import numpy as np
import cv2 as cv
import k4a

class kinect:
    # Kinect
    _device = None
    _capture = None

    # Depth
    _depth_image = None
    _depth = None

    def __init__(self):
        self.initialize()

    def __del__(self):
        self.finalize()

    def initialize(self):
        self.initialize_sensor()

    def finalize(self):
        self.finalize_sensor()

    def initialize_sensor(self):
        # Open Device
        self._device = k4a.Device.open()

        # Start Cameras
        device_config = k4a.DeviceConfiguration(
            color_format=k4a.EImageFormat.COLOR_BGRA32,
            color_resolution=k4a.EColorResolution.RES_720P,
            depth_mode=k4a.EDepthMode.NFOV_UNBINNED,
            camera_fps=k4a.EFramesPerSecond.FPS_30,
            synchronized_images_only=True
        )
        status = self._device.start_cameras(device_config)
        if status != k4a.EStatus.SUCCEEDED:
            raise IOError("failed start cameras!")

    def finalize_sensor(self):
        # Stop Cameras
        self._device.stop_cameras()

    def run(self):
        while True:
            self.update()
            self.draw()
            self.show()

            key = cv.waitKey(10)
            if key == ord('q'):
                break

    def update(self):
        self.update_frame()
        self.update_depth()

    def update_frame(self):
        # Capture Frame
        self._capture = self._device.get_capture(-1)
        if self._capture is None:
            raise IOError("failed get capture!")

    def update_depth(self):
        # Update Depth
        self._depth_image = self._capture.depth

    def draw(self):
        self.draw_depth()

    def draw_depth(self):
        # Draw Depth
        self._depth = self._depth_image.data

    def show(self):
        self.show_depth()

    def show_depth(self):
        # Scaling Depth
        self._depth = self._depth * (-255.0 / 5000.0) + 255.0
        self._depth = self._depth.astype(np.uint8)

        # Show Depth
        cv.imshow("depth", self._depth)


C:\Users\Nico\miniconda3\envs\masterthesis\lib\site-packages\k4a/../vendor/azure_kinect/windows/amd64/


In [2]:
try:
    kinect = kinect()
    kinect.run()
except Exception as e:
    print(e)

Exception ignored in: <function kinect.__del__ at 0x00000133F33BCAF0>
Traceback (most recent call last):
  File "C:\Users\Nico\AppData\Local\Temp\ipykernel_15844\1832276051.py", line 18, in __del__
  File "C:\Users\Nico\AppData\Local\Temp\ipykernel_15844\1832276051.py", line 24, in finalize
  File "C:\Users\Nico\AppData\Local\Temp\ipykernel_15844\1832276051.py", line 44, in finalize_sensor
AttributeError: 'NoneType' object has no attribute 'stop_cameras'


module 'k4a' has no attribute 'Device'


In [1]:
import traceback
import k4a, ctypes

C:\Users\Nico\miniconda3\envs\masterthesis\lib\site-packages\k4a/../vendor/azure_kinect/windows/amd64/


In [2]:
def VERIFY(result, error):
    if result != k4a.K4A_RESULT_SUCCEEDED:
        print(error)
        traceback.print_stack()
        sys.exit(1)

In [3]:
# Load device config
device_config = k4a.K4A_DEVICE_CONFIG_INIT_DISABLE_ALL
device_config.depth_mode = k4a.K4A_DEPTH_MODE_NFOV_UNBINNED
device = k4a.k4a_device_t()

In [4]:
# Open and start device
VERIFY(k4a.k4a_device_open(0, ctypes.byref(device)), "Open K4A Device failed!")
VERIFY(k4a.k4a_device_start_cameras(device, ctypes.byref(device_config)), "Start K4A cameras failed!")

In [5]:
# Depth sensor calibration
sensor_calibration = k4a.k4a_calibration_t()
VERIFY(k4a.k4a_device_get_calibration(device, device_config.depth_mode, k4a.K4A_COLOR_RESOLUTION_OFF, ctypes.byref(sensor_calibration)), "Get Sensor Calibration failed!")

In [7]:
# Body Tracker initialization
tracker = k4a.k4abt_tracker_t()
tracker_config = k4a.K4ABT_TRACKER_CONFIG_DEFAULT
VERIFY(k4a.k4abt_tracker_create(ctypes.byref(sensor_calibration), tracker_config, ctypes.byref(tracker)),"Initialize Tracker failed")

Initialize Tracker failed


  File "C:\Users\Nico\miniconda3\envs\masterthesis\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Nico\miniconda3\envs\masterthesis\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Nico\miniconda3\envs\masterthesis\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\Nico\miniconda3\envs\masterthesis\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\Nico\miniconda3\envs\masterthesis\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
    self.io_loop.start()
  File "C:\Users\Nico\miniconda3\envs\masterthesis\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\Nico\miniconda3\envs\masterthesis\lib\asyncio\base_events.py", line 601, in run_forever
    self._run_once()
  File "C:\Users\Nico\minico

SystemExit: 1

C:\Users\Nico\miniconda3\envs\masterthesis\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [21]:
sensor_capture = k4a.k4a_capture_t()
image = k4a.k4abt_frame_get_capture()
#get_capture_result = k4a.k4a_device_get_capture(device, ctypes.byref(sensor_capture), k4a.K4A_WAIT_INFINITE)
print(image)

TypeError: this function takes at least 1 argument (0 given)

In [8]:
frame_count = 0
while frame_count < 100:
        sensor_capture = k4a.k4a_capture_t()
        get_capture_result = k4a.k4a_device_get_capture(device, ctypes.byref(sensor_capture), k4a.K4A_WAIT_INFINITE)

        if get_capture_result == k4a.K4A_WAIT_RESULT_SUCCEEDED:
            frame_count += 1

            print("Start processing frame {}".format(frame_count))

            queue_capture_result = k4a.k4abt_tracker_enqueue_capture(tracker, sensor_capture, k4a.K4A_WAIT_INFINITE)

            k4a.k4a_capture_release(sensor_capture)

            if queue_capture_result == k4a.K4A_WAIT_RESULT_TIMEOUT:
                # It should never hit timeout when K4A_WAIT_INFINITE is set.
                print("Error! Add capture to tracker process queue timeout!")
                break
            elif queue_capture_result == k4a.K4A_WAIT_RESULT_FAILED:
                print("Error! Add capture to tracker process queue failed!")
                break

            body_frame = k4a.k4abt_frame_t()
            pop_frame_result = k4a.k4abt_tracker_pop_result(tracker, ctypes.byref(body_frame), k4a.K4A_WAIT_INFINITE)
            if pop_frame_result == k4a.K4A_WAIT_RESULT_SUCCEEDED:
                num_bodies = k4a.k4abt_frame_get_num_bodies(body_frame)
                print("{} bodies are detected!".format(num_bodies))

                for i in range(num_bodies):
                    body = k4a.k4abt_body_t()
                    body.id = k4a.k4abt_frame_get_body_id(body_frame, i)
                body_index_map = k4a.k4abt_frame_get_body_index_map(body_frame)
                if body_index_map:
                    k4a.k4a_image_release(body_index_map)
                else:
                    print("Error: Fail to generate bodyindex map!")

                k4a.k4abt_frame_release(body_frame)
            elif pop_frame_result == k4a.K4A_WAIT_RESULT_TIMEOUT:
                # It should never hit timeout when K4A_WAIT_INFINITE is set.
                print("Error! Pop body frame result timeout!")
                break
            else:
                print("Pop body frame result failed!")
                break
        elif get_capture_result == k4a.K4A_WAIT_RESULT_TIMEOUT:
            # It should never hit timeout when K4A_WAIT_INFINITE is set.
            print("Error! Get depth frame time out!")
            break
        else:
            print("Get depth capture returned error: {}".format(get_capture_result))

k4a.k4abt_tracker_shutdown(tracker)
k4a.k4abt_tracker_destroy(tracker)

Start processing frame 1
Error! Add capture to tracker process queue failed!


1019193280

In [2]:
# End and close Device
k4a.k4a_device_stop_cameras(device, ctypes.byref(device))
k4a.k4a_device_close(device)

NameError: name 'device' is not defined

In [3]:
import _k4a.py

ModuleNotFoundError: No module named '_k4a'